# 实现一个简单的 Tokenizer

在第二部分我们简单的介绍了一下分词器的作用，但当时我们使用的是 Qwen 已经训练好的分词器，本章将介绍如何训练一个简单的分词器。

---

## 准备工作

---

引入必要的包与定义必要的常量

In [ ]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, processors
from transformers import PreTrainedTokenizerFast, AutoTokenizer
from datasets import Dataset, load_dataset

import os
import json

**加载数据集**

本地 `.\dataset\wikitext-2-raw-v1` 目录下已经下载了训练所需的数据集，可以直接使用。资源下载地址：[huggingface-wikitext/wikitext-2-raw-v1](https://huggingface.co/datasets/Salesforce/wikitext/tree/main/wikitext-2-raw-v1)

In [ ]:
# 加载训练集
dataset = load_dataset("parquet", data_files={
    'train': './dataset/wikitext-2-raw-v1/train-00000-of-00001.parquet',
    'validation': './dataset/wikitext-2-raw-v1/validation-00000-of-00001.parquet',
    'test': './dataset/wikitext-2-raw-v1/test-00000-of-00001.parquet'
})

# 打印数据
# print(dataset['train'][:10])

# 构建数据迭代器
train_dataset = dataset['train']
batch_size = 1000
def batch_iterator():
    for i in range(0, len(train_dataset), batch_size):
        yield train_dataset[i : i + batch_size]["text"]

**保存 Tokenizer**

此代码保存为兼容 `AutoTokenizer` 的分词器格式。无需关注其实现。

In [ ]:
def token_decoder_dict(token_decoder, export_special = True):

    decoder_dict =  {
        "content" : token_decoder.content,
        "lstrip" : token_decoder.lstrip,
        "rstrip" : token_decoder.rstrip,
        "normalized" : token_decoder.normalized,
        "single_word" : token_decoder.single_word,
    }

    if export_special:
        decoder_dict["special"] = token_decoder.special

    return decoder_dict

def save_tokenizer(
        tokenizer,
        save_path,
        special_tokens,
        tokenizer_class = None,
        bos_token = None,
        eos_token = None,
        pad_token = None,
        unk_token = None
):
    # 创建保存目录
    os.makedirs(save_path, exist_ok=True)

    # 保存分词器
    tokenizer.save(f"{save_path}/tokenizer.json")

    # 保存词汇表
    with open(f"{save_path}/vocab.json", "w", encoding="utf-8") as f:
        json.dump(tokenizer.get_vocab(), f, ensure_ascii=False, indent=4)

    # 读取 JSON 文件
    with open(f"{save_path}/tokenizer.json", "r", encoding="utf-8") as file:
        tokenizer_data = json.load(file)

    # 保存 merges
    model_data = tokenizer_data['model']
    if 'merges' in model_data:
        with open(f"{save_path}/merges.txt", "w", encoding="utf-8") as f:
            for merge in model_data['merges']:
                f.write(f'{merge[0]} {merge[1]}\n')

    # 保存附加 token 表
    added_tokens = {}
    for added_token in special_tokens:
        added_tokens[added_token] = tokenizer.get_vocab()[added_token]
    with open(f"{save_path}/added_tokens.json", "w", encoding="utf-8") as f:
        json.dump(added_tokens, f, ensure_ascii=False, indent=4)

    # 获取分词器的 Token 解码器
    tokenizer_added_tokens_decoder = tokenizer.get_added_tokens_decoder()

    # 设置
    config = {}
    special_tokens_map = {}
    def add_special_token(name ,special_token):
        if special_token is not None:
            config[name] = special_token
            idx = tokenizer.get_vocab()[special_token]
            special_tokens_map[name] = token_decoder_dict(tokenizer_added_tokens_decoder[idx], False)

    add_special_token('bos_token', bos_token)
    add_special_token('eos_token', eos_token)
    add_special_token('unk_token', unk_token)
    add_special_token('pad_token', pad_token)

    if tokenizer_class is None:
        config["tokenizer_class"] = "PreTrainedTokenizerFast"
    else:
        config["tokenizer_class"] = tokenizer_class

    if special_tokens is not None:
        added_tokens_decoder = {}
        for key in tokenizer_added_tokens_decoder:
            token_decoder = tokenizer_added_tokens_decoder[key]
            added_tokens_decoder[key] = token_decoder_dict(token_decoder)
        config['added_tokens_decoder'] = added_tokens_decoder

    with open(f"{save_path}/special_tokens_map.json", "w", encoding="utf-8") as f:
        json.dump(special_tokens_map, f, ensure_ascii=False, indent=4)

    with open(f"{save_path}/tokenizer_config.json", "w", encoding="utf-8") as f:
        json.dump(config, f, ensure_ascii=False, indent=4)


## Tokenizer 算法

---

### 1. Byte-Pair Encoding (BPE)

**原理：** 通过迭代合并高频字符对生成子词词汇表，例如将 `"l o w"` 和 `"l o w e r"` 合并为 `low` 和 `lower`。

**特点：**
- 贪心合并，适合高频词和常见子词。
- 词汇表大小可自定义（例如 10k-100k）。

**应用模型：** GPT 系列（如 GPT-2、GPT-3）、RoBERTa。

**优势：**
- 高效处理未知词（如 `"chatting"` → `"chat" + "ting"`）。
- 简单且计算速度快。

**使用 BPE 算法训练 Tokenizer**

In [ ]:
unk_token = '<unk>'
pad_token = '<pad>'
bos_token = '<bos>'
eos_token = '<eos>'
vocab_size = 20000

# 特殊 Token
special_tokens = [unk_token, pad_token, bos_token, eos_token]

# 实例化一个 BPE 分词器
tokenizer = Tokenizer(models.BPE(unk_token=unk_token))

# 定义 BpeTrainer
trainer = trainers.BpeTrainer()

# 设置预分词器，这里使用简单的空格分词
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# 添加特殊 token
tokenizer.add_special_tokens(special_tokens)

# 训练分词器
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

# 保存训练好的分词器
save_tokenizer(
    tokenizer,
    save_path ="tokenizer/bpe-tokenizer",
    special_tokens = special_tokens,
    bos_token = bos_token,
    eos_token = eos_token,
    pad_token = pad_token,
    unk_token = unk_token,
)

# 加载tokenizer
# bpe_tokenizer = Tokenizer.from_file("./tokenizer/bpe-tokenizer/tokenizer.json")

**测试 BEP Tokenizer**

In [ ]:
bpe_tokenizer = AutoTokenizer.from_pretrained("./tokenizer/bpe-tokenizer", trust_remote_code=True)

# 编解码测试
token_encode = bpe_tokenizer("<bos>How are you ?<eos>", padding="max_length", max_length=10)
print('token_encode: ', token_encode)

token_seq = bpe_tokenizer.decode(token_encode['input_ids'])
print('token_seq: ', token_seq)

## 2. WordPiece

WordPiece 是一种基于子词（subword）的分词算法，由谷歌在2016年提出，是BERT等主流预训练模型的分词核心。其基本思想是将单词拆分成更小的子词单元，以减少词表的大小，同时解决未登录词（OOV）的问题。以下是WordPiece分词器算法的简要介绍：

**原理**
- **核心思想：** WordPiece算法通过不断合并相邻的子词对来构建词表，每次选择合并的子词对是基于概率计算的，即选择合并后能最大化语言模型概率的子词对。具体来说，计算两个相邻子词的互信息值（PMI），选择具有最大PMI值的子词对进行合并。
- **与BPE算法的区别：** BPE算法是基于子词对的频率来选择合并的子词对，而WordPiece算法则更进一步，考虑了子词对在语言模型中的关联性，使得合并后的子词对在语义上更连贯。

**分词过程**

- **训练阶段：**
    1. **初始化：** 将语料库中的所有单词以字符为单位进行拆分，每个字符作为一个子词，初始化词表为所有可能的字符集合。
    2. **统计与合并：** 遍历每个单词中所有可能的子词对，统计它们的出现频数，计算每个子词对的得分（如PMI值），选择得分最高的子词对进行合并，将合并后的子词加入词表，并记录合并规则。
    3. **迭代：** 重复上述统计与合并步骤，直到词表达到预设的大小。
- **分词阶段：** 利用训练好的词表，对输入的单词进行正向最长匹配分词，即不断寻找单词中的最长前缀子词。

**优势**
- **减少词表大小：** 通过将单词拆分成子词，可以有效减少词表的大小，同时保留常见词的完整性。
- **解决未登录词问题：** 对于未在训练数据中出现的单词，可以将其拆分成已存在的子词组合，从而避免了未登录词的问题。
- **提高模型泛化能力：** WordPiece算法能够更好地捕捉子词之间的语义关联，有助于提高模型在不同语料上的泛化能力。

WordPiece分词器算法在自然语言处理领域得到了广泛应用，特别是在基于Transformer的预训练模型中，为模型的训练和应用提供了有效的文本预处理手段。

In [ ]:
unk_token = '<unk>'
pad_token = '<pad>'
bos_token = '<bos>'
eos_token = '<eos>'

# 特殊 Token
special_tokens = [unk_token, pad_token, bos_token, eos_token]

# 实例化一个 BPE 分词器
tokenizer = Tokenizer(models.WordPiece(unk_token=unk_token))

# 定义 BpeTrainer 并设置参数
trainer = trainers.WordPieceTrainer(
    special_tokens = special_tokens,
    vocab_size = vocab_size,
    min_frequency = 2,
    show_progress = True
)

# 设置预分词器，这里使用简单的空格分词
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# 添加特殊 token
tokenizer.add_special_tokens(special_tokens)

# 训练分词器
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

# 保存训练好的分词器
save_tokenizer(
    tokenizer,
    save_path ="tokenizer/wordpiece-tokenizer",
    special_tokens = special_tokens,
    bos_token = bos_token,
    eos_token = eos_token,
    pad_token = pad_token,
    unk_token = unk_token,
)

**测试 WordPiece Tokenizer**

In [ ]:
bpe_tokenizer = AutoTokenizer.from_pretrained("./tokenizer/wordpiece-tokenizer", trust_remote_code=True)

# 编解码测试
token_encode = bpe_tokenizer("<bos>How are you ?<eos>", padding="max_length", max_length=10)
print('token_encode: ', token_encode)

token_seq = bpe_tokenizer.decode(token_encode['input_ids'])
print('token_seq: ', token_seq)